In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import open3d as o3d
import numpy as np

In [3]:
torch.manual_seed(1)

In [4]:
data = []
pcd = o3d.io.read_point_cloud("realsensedata.ply")
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                             ransac_n=3,
                                             num_iterations=1000)
[pa, pb, pc, pd] = plane_model
print(f"Plane equation: {pa:.2f}x + {pb:.2f}y + {pc:.2f}z + {pd:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

outlier_cloud = pcd.select_by_index(inliers, invert=True)

###############2
plane2_model, inliers2 = outlier_cloud.segment_plane(distance_threshold=0.01,
                                                         ransac_n=3,
                                                         num_iterations=1000)
[pa2, pb2, pc2, pd2] = plane2_model
print(f"Plane equation2: {pa2:.2f}x + {pb2:.2f}y + {pc2:.2f}z + {pd2:.2f} = 0")
    
inlier2_cloud = outlier_cloud.select_by_index(inliers2)
inlier2_cloud.paint_uniform_color([1.0, 0, 0])

outlier2_cloud = outlier_cloud.select_by_index(inliers2, invert=True)

################3
plane3_model, inliers3 = outlier2_cloud.segment_plane(distance_threshold=0.01,
                                                          ransac_n=3,
                                                          num_iterations=1000)
[pa3, pb3, pc3, pd3] = plane3_model
print(f"Plane equation3: {pa3:.2f}x + {pb3:.2f}y + {pc3:.2f}z + {pd3:.2f} = 0")
inlier3_cloud = outlier2_cloud.select_by_index(inliers3)
inlier3_cloud.paint_uniform_color([1.0, 0, 0])
    
data.append(inlier_cloud)
data.append(inlier2_cloud)
data.append(inlier3_cloud)
    
p1 = np.asarray(data[0].points)
p2 = np.asarray(data[1].points)
pzero = np.zeros((p1.shape[0],3),dtype=float)

##Y_TRAIN 
#P2 -> P1 평면 사이거리
#P1 -> P2 평면 사이거리 
for i in range(0,p2.shape[0]):
    for j in range(0,3):
        pzero[i][j] = p2[i][j]


x1_traIn = torch.FloatTensor(p1)
x2_train = torch.FloatTensor(p2)
print(pzero.shape)
print(p1.shape)
print(p2.shape)
print(pa,pa2,pa3)

w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)





Plane equation: -0.03x + 0.92y + 0.39z + -0.93 = 0
Plane equation2: 0.95x + -0.23y + 0.22z + -1.17 = 0
Plane equation3: -0.40x + -0.62y + 0.67z + -2.20 = 0
(147548, 3)
(147548, 3)
(62199, 3)
-0.03035043602492938 0.9460683532461426 -0.39806572493574394


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import open3d as o3d
import numpy as np
import math 
import copy

data = []
pcd = o3d.io.read_point_cloud("best/centerslowdata/fragment_014.ply")
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                             ransac_n=3,
                                             num_iterations=1000)
[pa, pb, pc, pd] = plane_model
print(f"Plane equation: {pa:.2f}x + {pb:.2f}y + {pc:.2f}z + {pd:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

outlier_cloud = pcd.select_by_index(inliers, invert=True)

###############2
plane2_model, inliers2 = outlier_cloud.segment_plane(distance_threshold=0.01,
                                                         ransac_n=3,
                                                         num_iterations=1000)
[pa2, pb2, pc2, pd2] = plane2_model
print(f"Plane equation2: {pa2:.2f}x + {pb2:.2f}y + {pc2:.2f}z + {pd2:.2f} = 0")
    
inlier2_cloud = outlier_cloud.select_by_index(inliers2)
inlier2_cloud.paint_uniform_color([0, 1.0, 0])

outlier2_cloud = outlier_cloud.select_by_index(inliers2, invert=True)

################3
plane3_model, inliers3 = outlier2_cloud.segment_plane(distance_threshold=0.01,
                                                          ransac_n=3,
                                                          num_iterations=1000)
[pa3, pb3, pc3, pd3] = plane3_model
print(f"Plane equation3: {pa3:.2f}x + {pb3:.2f}y + {pc3:.2f}z + {pd3:.2f} = 0")
inlier3_cloud = outlier2_cloud.select_by_index(inliers3)
inlier3_cloud.paint_uniform_color([0, 0, 1.0])


data.append(inlier_cloud)
data.append(inlier2_cloud)
data.append(inlier3_cloud)

a1,b1,c1,d1 = (pa + pa2, pb + pb2, pc + pc2, pd+pd2) #3
a2,b2,c2,d2 = (pa2 - (pa3 * pa2/pa3), pb2 - (pb3 * pa2/pa3), pc2 - (pc3 * pa2/pa3), pd2 - (pd3 * pa2/pa3))
print("a2!!!!:",a2)
#print("3번식: ", a)
#print("4번식: ", b)
#print(f"Crossing lines#3: {a1:.2f}x + {b1:.2f}y + {c1:.2f}z + {d1:.2f} = 0")
#print(f"Crossing lines#4: {b2:.2f}y + {c2:.2f}z + {d2:.2f} = 0")
print(f"Crossing lines: {a1:.2f}x + {b1-b2:.2f}y + {c1-c2:.2f}z + {d1-d2:.2f} = 0")
gac1 = abs(pa * pa2 + pb * pb2 + pc * pc2) / math.sqrt(math.pow(pa,2) + math.pow(pb,2) + math.pow(pc,2)) * math.sqrt(math.pow(pa2,2) + math.pow(pb2,2) + math.pow(pc2,2))
gac2 = abs(pa * pa3 + pb * pb3 + pc * pc3) / math.sqrt(math.pow(pa,2) + math.pow(pb,2) + math.pow(pc,2)) * math.sqrt(math.pow(pa3,2) + math.pow(pb3,2) + math.pow(pc3,2))
gac3 = abs(pa2 * pa3 + pb2 * pb3 + pc2 * pc3) / math.sqrt(math.pow(pa2,2) + math.pow(pb2,2) + math.pow(pc2,2)) * math.sqrt(math.pow(pa3,2) + math.pow(pb3,2) + math.pow(pc3,2))
print(gac1)
print(gac2)
print(gac3)

print("inlier cloud type!!!:",inlier_cloud)
planbox1 = inlier_cloud.get_oriented_bounding_box()
planbox2 = inlier2_cloud.get_oriented_bounding_box()
planbox3 = inlier3_cloud.get_oriented_bounding_box()


planboxpoint2 = np.asarray(planbox2.get_box_points())
planboxpoint3 = np.asarray(planbox3.get_box_points())

#print("planbox1:",planbox1)                                                                        
#print("planbox1 center pointcloud",planbox1.get_center())
#print("planbox1 max pointcloud",planbox1.get_max_bound())
#print("planbox1 min pointcloud",planbox1.get_min_bound())


print("planbox2 edge points:",planboxpoint2)
print("planbox3 edge points:",planboxpoint3)

print("inlier2_cloud:",np.asarray(inlier2_cloud.points))
print("inlier3_cloud:",np.asarray(inlier3_cloud.points))
print("plan2 eqation:", pa2, pb2, pc2, pd2)

crossline = [a2, b2, c2, d2]

print("plane1 and plane2 crossline:", crossline)
cnt = 0
m = []
for i in range(0,len(inlier2_cloud.points)):
    x = inlier2_cloud.points[i][0]
    y = inlier2_cloud.points[i][1]
    z = inlier2_cloud.points[i][2]
    result = (a2* x) + (b2 * y) + (c2 * z) + d2 
    if round(result) == 0.0:
        m.append(x)
        m.append(y)
        m.append(z)
        
data = np.array(m)
data = data.reshape(-1,3)
print(data)
pc_array = np.asarray(data)
pc_new = o3d.geometry.PointCloud()
pc_new.points = o3d.utility.Vector3dVector(data)
pc = o3d.io.write_point_cloud("pc2pcd.pcd", pc_new)
pc2 = o3d.io.read_point_cloud("pc2pcd.pcd")
pc2.paint_uniform_color([0.0 , 0.0, 0.0])
#print("pc_new.points!!!!!!!:",pc_new.points)
#print("pc!!!!!!!:",pc)
#print("pc2!!!!!!!:",pc2)
planboxdata = pc2.get_oriented_bounding_box()

#print("m shape!!:",m.reshape(-1,3))


vis = o3d.visualization.Visualizer()    
vis.create_window()

vis.add_geometry(planbox1)
#vis.add_geometry(planbox2)
#vis.add_geometry(planbox3)

vis.add_geometry(planboxdata)
vis.add_geometry(pc2)

vis.add_geometry(inlier_cloud)
vis.add_geometry(inlier2_cloud)
vis.add_geometry(inlier3_cloud)

o3d.visualization.ViewControl.set_zoom(vis.get_view_control(), 0.8)
vis.run()
vis.destroy_window()



Plane equation: 0.46x + -0.52y + 0.72z + -1.45 = 0
Plane equation2: 0.97x + 0.22y + -0.04z + 1.20 = 0
Plane equation3: -0.06x + 0.87y + 0.49z + -1.11 = 0
a2!!!!: 0.0
Crossing lines: 1.44x + -15.75y + -7.76z + 17.93 = 0
0.305085535179296
0.12508342752293689
0.11650720181112978
inlier cloud type!!!: PointCloud with 83611 points.
planbox2 edge points: [[-1.47975159  1.1092071   0.21890824]
 [-1.14938261  0.10802232  2.70914894]
 [-1.21705775 -0.18014244 -0.33431661]
 [-1.45707481  1.11424847  0.21792666]
 [-0.86401198 -1.17628586  2.15494252]
 [-1.19438096 -0.17510107 -0.33529819]
 [-1.12670582  0.11306369  2.70816736]
 [-0.88668876 -1.18132722  2.15592409]]
planbox3 edge points: [[-1.07774606 -0.22230483  2.57853373]
 [ 1.06021351  0.28720846  1.90686261]
 [-1.87123086  0.73385344  0.77815399]
 [-1.07661359 -0.24034455  2.56845395]
 [ 0.26786118  1.22532701  0.09640309]
 [-1.87009839  0.71581372  0.76807421]
 [ 1.06134598  0.26916873  1.89678284]
 [ 0.26672871  1.24336673  0.10648287]]
i

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import open3d as o3d
import numpy as np
import math 
import copy
pc_new = o3d.geometry.PointCloud()
print(pc_new)



In [ ]:
equation = (1.43 * x) + (0.34 * y) + (-0.07 * z) + 1.76
while True:
    cnt = -2
    cnt2 = 2
    step = 0.001
    
    if (1.43 * x) + (0.34 * y) + (-0.07 * z) + 1.76

